In [1]:
import sys

sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import hashlib
from ast import literal_eval
from pathlib import Path
from functools import reduce

from src.utils import find_meta_category
from src.feature_extractor import sample_feature_combinations

## Download prerequisite files

Fetch all the results and feature values


In [3]:
# You can get the experiments file here: 01J6KF3JRCATRJQ9CPJTRV5VBM (https://beaker.org/ds/01J6KF3JRCATRJQ9CPJTRV5VBM/details)
!echo "Fetching experiments list..."
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix experiments.txt
!echo "Fetching extracted features..."
!mkdir features/
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix features/ 
#!beaker dataset fetch 01J6KFVCRCTYHCZDR0XNK0G9HT --prefix features/
!echo "Fetching helpsteer2 dataset"
!beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM
!echo "Collating all evaluation results"
%run ../../scripts/fetch_evals_rewardbench.py --output_file results.csv --gpt4_threshold_score 0.658 --experiment_prefix rm-eval-helpsteer2 --experiments_file experiments.txt

Fetching experiments list...
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 73.77 KiB  ⠙  
Files: 1          ✔  
Bytes: 73.77 KiB  ✔  
Files: 1          ✔  
Bytes: 73.77 KiB  ✔  
Completed in 100ms: 436.4 KiB/s, 6 files/s
Fetching extracted features...
mkdir: features/: File exists
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 2          ⠙  
Bytes: 75.1 MiB   ⠙  
Files: 7          ⠹  
Bytes: 263.9 MiB  ⠹  
Files: 13         ⠸  
Bytes: 489.2 MiB  ⠸  
Files: 16         ⠼  
Bytes: 602.9 MiB  ⠼  
Files: 16         ✔  
Bytes: 602.9 MiB  ✔  
Completed in 500ms: 1.087 GiB/s, 30 files/s
Fetching helpsteer2 dataset
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 70.58 MiB  ⠙  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Completed in 200ms: 322.2 MiB/s, 5 files/s
Collating all evaluation results
2024-09-03 20:41:23 - INFO - root - Logged-in as ljm (ljm@allenai.org)
2024-09-03 20:41:25 - INFO - root - Found 287 experiments that match 'rm-eva

100%|██████████| 287/287 [00:55<00:00,  5.18it/s]

2024-09-03 20:42:20 - INFO - root - Computing category scores...
2024-09-03 20:42:20 - INFO - root - Deriving features from the experiments file: experiments.txt
2024-09-03 20:42:20 - INFO - root - Will attempt merge via feature hash
2024-09-03 20:42:20 - INFO - root - Creating labels in column 'label' with GPT-4 threshold '0.658'
2024-09-03 20:42:20 - INFO - root - Saving 128 results to results.csv
2024-09-03 20:42:20 - INFO - root - Saved on results.csv


Collate feature set for all instances


In [4]:
LEXICAL_FEATS_PATH = Path("features")
DATASET_PATH = Path("helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl")


def get_dataset_features(
    feature_path=LEXICAL_FEATS_PATH, dataset_path=DATASET_PATH
) -> "pd.DataFrame":
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]
    lexical_feature_files = [
        file
        for file in feature_path.glob("*.jsonl")
        if any(file.stem in feat for feat in lexical_features)
    ]
    lexical_feats_df = reduce(
        lambda left, right: left.merge(
            right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
        ),
        [pd.read_json(file, lines=True) for file in lexical_feature_files],
    )

    df = pd.read_json(dataset_path, lines=True).rename(columns={"prompt_hash": "id"})
    finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
        columns=["prompt", "completion_a", "completion_b"]
    )

    # Hacky way for token_len_difference
    finaldf = finaldf.rename(columns={"token_len_diff": "token_len_difference"})
    return finaldf

In [5]:
results_df = pd.read_csv("results.csv").dropna()
features_df = get_dataset_features()
print(len(results_df))

127


## Get proportion of instances that fulfill the conditions

1. For each row, get features that were activated
2. Then for each activated feature, we get the proportion by looking at the feature dataframe.
3. The proportion is computed as: `number_of_instance_that_fulfill_a_single_condition` / `total_number_of_instances`


In [6]:
# Inspect nan columns
rows_with_nan = features_df[features_df.isna().any(axis=1)]
nan_columns = rows_with_nan.columns[rows_with_nan.isna().any()]
df_nan_columns = rows_with_nan[nan_columns]
df_nan_columns

,expertise_level,format_constraints
289,None,[]
1317,expert domain knowledge,None
4613,basic domain knowledge,None
4734,general public,None


So what you're going to do instead, is to take the binary_cols, and then for each element of that binary_cols, you compute the "weight"


In [7]:
def compute_instances(feat: str, features_df: "pd.DataFrame") -> float:
    total = len(features_df)
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]

    if feat.split("__")[0] in lexical_features:
        feat_name, value = feat.split("__")
        min_val_str, max_val_str = value.split("|")
        min_val, max_val = float(min_val_str.split("=")[1]), float(
            max_val_str.split("=")[1]
        )
        return features_df[feat_name].between(min_val, max_val).mean()
    else:
        # Parse the feature
        feat_name, value = feat.split("=")
        meta_category = find_meta_category(feat_name)
        if meta_category == "scalar":
            v = value.replace("_", " ")
            return features_df[feat_name].value_counts().get(v) / total
        elif meta_category == "closed_set":
            v = value.replace("_", " ")
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if v in listval else 0 for listval in list_of_values]) / total
        elif meta_category == "open_set":
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if listval else 0 for listval in list_of_values]) / total

        return find_meta_category(feat_name)


feats = results_df.columns[results_df.isin([0, 1]).all()]  # get binary columns
feat_map = {
    feat: compute_instances(feat, features_df) for feat in feats if feat != "label"
}

ratio_df = results_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
)

# Regressor training


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## Train LinearRegressor


In [9]:
polyfit = False
binary = False

X = ratio_df[list(feat_map.keys())]
y = ratio_df["Overall"]
if binary:
    X = (X > 0).astype(int)

if polyfit:
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_poly, y, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )


print(f"Train size: {len(X_train)}, test size: {len(X_test)}")

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# print(f"Feature names: {poly.get_feature_names_out(X.columns)}")
print(f"Mean Squared Error: {mse}")
print(f"Coeeficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Train size: 101, test size: 26
Mean Squared Error: 0.0011799988127666999
Coeeficients: [-1.69139184e-01  5.39745677e-03 -4.83589839e-02 -2.50702320e-02
 -9.20540054e-02  1.13969225e-02 -3.73776476e-02  4.85162352e-02
 -1.71390841e-01  2.37269423e-02  4.98207203e-03 -2.68490014e-03
 -5.33340856e-02 -6.41223597e-02 -9.68918665e-03 -2.25369590e-01
  9.00947691e-02 -6.98322731e-02  1.15067201e-02 -1.71432920e-12
 -1.88090862e-12 -6.59541866e-13  1.52877710e-13  3.31004668e-12
 -1.20331003e+01 -1.81850530e-01 -2.89864605e-01 -2.94473646e-02
  1.02749678e-01  1.91846539e-13 -3.77475828e-15 -2.32642237e-01
 -8.46659730e-03 -1.35201316e-02 -4.57418474e-01 -2.14973263e+00
 -8.39792306e-01  0.00000000e+00 -4.43099315e-03 -3.55271368e-15
 -2.59171788e-01  1.95399252e-14  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.78806648e-01  0.00000000e+00
 -7.38329999e-01  0.00000000e+00 -1.00666263e+01  0.00000000e+00
  0.00000000e+00  4.23964063e-01  3.90277829e+00  0.00000000e+00
  5

In [10]:
if not polyfit:
    feature_importance = pd.DataFrame(
        {"Feature": X.columns, "Coefficient": model.coef_}
    )

    # Calculate absolute importance for easier comparison
    feature_importance["Absolute_Coefficient"] = np.abs(
        feature_importance["Coefficient"]
    )

    # Sort by absolute coefficient value
    feature_importance = feature_importance.sort_values(
        by="Absolute_Coefficient", ascending=False
    )
    feature_importance.head(10)
else:
    print(
        "Feature importance is not possible with polynomial features (hard to interpret)"
    )

## Simulation


In [11]:
from tqdm import tqdm_notebook

In [12]:
_, combinations = sample_feature_combinations(
    meta_analyzer_n_samples=2000, max_number=10
)

10it [00:00, 79891.50it/s]
45it [00:00, 102300.10it/s]
120it [00:00, 75800.67it/s]
210it [00:00, 65477.54it/s]
252it [00:00, 51564.28it/s]
210it [00:00, 42786.55it/s]
120it [00:00, 37153.35it/s]
45it [00:00, 28233.91it/s]
10it [00:00, 19266.44it/s]
1it [00:00, 10782.27it/s]

2024-09-03 20:42:23 - INFO - root - Adding meta analyzer features



10it [00:00, 108942.96it/s]
45it [00:00, 83923.38it/s]
120it [00:00, 78373.79it/s]
210it [00:00, 63153.64it/s]
252it [00:00, 51281.58it/s]
210it [00:00, 42083.32it/s]
120it [00:00, 19333.04it/s]
45it [00:00, 29662.69it/s]
10it [00:00, 17396.53it/s]
1it [00:00, 14122.24it/s]


In [13]:
sim_df = pd.DataFrame(0, index=np.arange(len(combinations)), columns=X.columns)
for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):
    activated_feats = []
    for feat in combination:
        if "analyzer" in feat:
            feature_name_str, value_str = feat.split("::")[1].split("|")
            feature_name, value = (
                feature_name_str.split("=")[-1],
                value_str.split("=")[-1],
            )
            activated_feats.append(f"{feature_name}={value}")
        else:
            activated_feats.append(feat.replace("::", "__"))
    sim_df.loc[idx, activated_feats] = 1
sim_df = sim_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
).dropna(axis=1, how="any")

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_24031/4023816742.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):


  0%|          | 0/4069 [00:00<?, ?it/s]

In [14]:
sim_results = sim_df.copy(deep=True)
sim_results["activated_features"] = sim_results.apply(
    lambda row: [col for col in sim_results.columns if row[col] != 0], axis=1
)
sim_results["pred"] = model.predict(sim_df)
sim_results = sim_results.sort_values(by="pred", ascending=False).reset_index(drop=True)
sim_results[["activated_features", "pred"]].head(20)

,activated_features,pred
0,"[complexity_of_intents=simple, languages=Engli...",0.764134
1,"[complexity_of_intents=simple, languages=English]",0.760119
2,"[complexity_of_intents=simple, languages=Engli...",0.755809
3,"[complexity_of_intents=simple, cosine_sim__min...",0.753833
4,"[complexity_of_intents=simple, cosine_sim__min...",0.751735
5,[complexity_of_intents=simple],0.748626
6,[complexity_of_intents=simple],0.748626
7,[complexity_of_intents=simple],0.748626
8,[complexity_of_intents=simple],0.748626
9,[complexity_of_intents=simple],0.748626


In [26]:
top_n = 100
human_score = 0.715
better_than_humans = sim_results[sim_results["pred"] > human_score]
top_combinations = (
    better_than_humans.activated_features.head(top_n).drop_duplicates().to_list()
)
print(top_combinations)

[['complexity_of_intents=simple', 'languages=English', 'open_endedness=no', 'safety_concern=moderate'], ['complexity_of_intents=simple', 'languages=English'], ['complexity_of_intents=simple', 'languages=English', 'safety_concern=safe'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.33|max_val=0.67'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.33|max_val=0.67', 'rouge__min_val=0.0|max_val=0.33'], ['complexity_of_intents=simple'], ['bertscore__min_val=0.67|max_val=1.0', 'complexity_of_intents=simple', 'cosine_sim__min_val=0.67|max_val=1.0', 'entity_sim__min_val=0.0|max_val=0.33', 'rouge__min_val=0.33|max_val=0.67'], ['complexity_of_intents=simple', 'cosine_sim__min_val=0.67|max_val=1.0', 'languages=English', 'rouge__min_val=0.67|max_val=1.0'], ['complexity_of_intents=simple', 'format_constraints=1', 'languages=English', 'open_endedness=no', 'safety_concern=moderate'], ['complexity_of_intents=simple', 'format_constraints=1', 'languages=English', 'open_endedness=no']

So now you have determined 10 feature combinations that seem to work well. The next step is to train RMs and evaluate them.


In [28]:
from beaker import Beaker, ExperimentSpec
from copy import deepcopy

In [29]:
spec = ExperimentSpec.from_file("../../beaker/template.yml")
exp_spec = deepcopy(spec)
template_task = exp_spec.tasks.pop(0)

new_tasks = []
for idx, combination in enumerate(top_combinations):
    feats_to_run = []
    for feat in combination:
        if "min_val" in feat:
            if "token_len_difference" in feat:
                feat = feat.replace("difference", "diff")
            feats_to_run.append(feat.replace("__", "::"))
        else:
            feat_name, value = feat.split("=")
            category = find_meta_category(feat_name)
            if category == "closed_set":
                key = "constraints"
            elif category == "scalar":
                key = "value"
            elif category == "open_set":
                key = "check_for_existence"
            feats_to_run.append(f"{category}::feature_name={feat_name}|{key}={value}")
    # Create beaker task
    task = deepcopy(template_task)
    task.name = f"get-features-datamodel-{idx}"
    task.arguments.extend(["--features"] + feats_to_run)
    new_tasks.append(task)

exp_spec.tasks = new_tasks
exp_spec.validate()
exp_spec.to_file("experiments.yml")

Get finished jobs and download the subsets and create an `experiments.txt` file


In [30]:
# experiment_id = "01J6TS47Q2KNKYRCYHC8A0DE4B"
# experiment_id = "01J6WDKDPQCM92REXJ1VCNJ0NW"
experiment_id = "01J6XJSWMSAM2ARJ1WXAP6PV8T"
top_subsets_dir = Path("top_n_subsets")
top_subsets_dir.mkdir(parents=True, exist_ok=True)
experiments_file = top_subsets_dir / "top_n_subsets_experiments.txt"
beaker = Beaker.from_env("ai2/ljm-oe-adapt")
experiment = beaker.experiment.get(experiment_id)

experiment_ids = []
for job in tqdm_notebook(experiment.jobs):
    if job.is_done:
        # Get output
        dataset_id = job.execution.result.beaker
        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="data/",
            quiet=True,
        )

        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="experiments.txt",
            quiet=True,
        )

        with open(top_subsets_dir / "experiments.txt", "r") as f:
            data = f.read().splitlines()
            id = data[0]
            experiment_ids.append(id)

print(experiment_ids)
with open(experiments_file, "a") as f:
    for id in set(experiment_ids):
        f.write("\n" + id)

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_24031/76539003.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for job in tqdm_notebook(experiment.jobs):


  0%|          | 0/65 [00:00<?, ?it/s]

In [19]:
len(set(experiment_ids))

13